In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import jupyterlab_dash
import dash
import dash_html_components as html

from pathlib import Path

import plotly.express as px

import plotly.graph_objs as go

from dash.dependencies import Input, Output

import pandas as pd

In [ ]:
base_dir = Path().cwd().parent
processed_dir = base_dir/'data'/'processed'/'daily_report'

march30 = pd.read_csv(processed_dir/'03-30-2020.csv')
march31 = pd.read_csv(processed_dir/'03-31-2020.csv')

march30.loc[:, 'date'] = pd.to_datetime('03-30-2020')
march31.loc[:, 'date'] = pd.to_datetime('03-31-2020')

df = march30.append(march31)

In [ ]:
df.head()

In [ ]:
days = pd.date_range('03/01/2020', '04/02/2020', normalize=True)
days = days.strftime('%m-%d-%Y')

In [ ]:
processed_dir

In [ ]:
days[0]

In [ ]:
processed_dir/f'{day}.csv'

In [ ]:
base_dir = Path().cwd().parent
processed_dir = base_dir/'data'/'processed'/'daily_report'

df = pd.DataFrame()
for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = pd.to_datetime(day)
    df = df.append(temp)

In [ ]:
def get_date_slider_marks(daterange=None):
"""
    daterange pd.Series
"""
    daterange.unique()
    
    

In [ ]:
df['date']

In [ ]:
pd.to_datetime(day)

## generate dataframe

In [ ]:
df = pd.DataFrame()  

for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = pd.to_datetime(day)
    df = df.append(temp)

## generate date picker

In [ ]:
def get_date_picker():   
    return dcc.DatePickerSingle(
        id='date-picker',
        min_date_allowed=dt(2020,3,1),
        max_date_allowed=dt(2020,4,2),
        initial_visible_month=dt(2020,3,15),
        date=str(dt(2020, 3, 15))
    )

## generate table

In [ ]:
def generate_table(df, max_rows=5):
    
    columns = ['country', 'confirmed']
    
    df = df[columns].copy()
    df = df.sort_values(by=columns[1], ascending=False)
    
    fig = go.Figure(
        data=
        [
            go.Table(
                header=dict(values=[c.capitalize() for c in df[columns].columns],
                fill_color='lightgrey',
                align='left'),
                cells=dict(
                    values=[df[columns[0]], 
                            df[columns[1]]],
                    fill_color='white',
                    align='left'),
            )
        ]
    )
    
    return fig

In [ ]:
from datetime import datetime as dt
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import re

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    
    # set page header
    html.H4(children='COVID-Dashboard')
    ,
    
    # set date picker via calendar    
    html.Div(children=[
        get_date_picker(),
    ],
        # style={'width': '49%', 'display': 'inline-block'}
    ),
    
    html.Div(children=[
        
        # set map graph 
        html.Div(
            dcc.Graph(id='graph-map'), 
            style={'width': '49%', 'display': 'inline-block'}
        ),
        
        # set country info table
        html.Div(
            dcc.Graph(id='table-info', figure=generate_table(df)), 
            style={'width': '49%', 'display': 'inline-block'}
        )
    ],
    ),
])

@app.callback(
    Output('graph-map', 'figure'),
    [Input('date-picker', 'date')])
def update_output(date):

    df_map = df[df['date'] == pd.to_datetime(date)]
    trace = go.Choropleth(
        locations=df_map['country'],
        locationmode='country names',
        z=df_map['confirmed'],
        text=df_map['country'],
        autocolorscale=False,
        colorscale="YlOrRd",
    )
    
    
    string_prefix = 'You have selected: '
    if date is not None:
        date = dt.strptime(re.split('T| ', date)[0], '%Y-%m-%d')
        date_string = date.strftime('%B %d, %Y')
        
    map_update = {
        'data': [trace], 
        'layout': {
            'title_text':'2014 Global GDP',
            'geo': {
                'showframe': True,
                'showcoastlines': True,
                'projection_type': 'equirectangular'
            }
        }
    }
        

    return (map_update)

viewer = jupyterlab_dash.AppViewer()
viewer.show(app)